In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random
import networkx as nx
import folium
from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString, MultiLineString, Polygon
import webbrowser
import json
import geopandas as gpd
from math import radians, sin, cos, sqrt, atan2
from folium.plugins import MarkerCluster, FeatureGroupSubGroup


In [2]:

# 폴더 경로 설정
folder_path = '../processed_geojson_data'

# GeoJSON 파일 읽기
def load_geojson_files(folder_path):
    geojson_data = {}
    
    # 폴더 내 모든 파일 탐색
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.geojson'):
            file_path = os.path.join(folder_path, file_name)
            try:
                gdf = gpd.read_file(file_path)
                geojson_data[file_name] = gdf
                print(f"{file_name} 로드 완료. 데이터 크기: {gdf.shape}")
            except Exception as e:
                print(f"{file_name} 로드 중 오류 발생: {e}")
    
    return geojson_data

# 데이터 불러오기
geo_df = load_geojson_files(folder_path)

# 데이터 확인
for name, gdf in geo_df.items():
    print(f"파일명: {name}, 열 목록: {list(gdf.columns)}")


bus_stops.geojson 로드 완료. 데이터 크기: (11290, 7)
legal_boundary.geojson 로드 완료. 데이터 크기: (25, 3)
parking_lot.geojson 로드 완료. 데이터 크기: (8539, 4)


road.geojson 로드 완료. 데이터 크기: (67669, 12)
school.geojson 로드 완료. 데이터 크기: (3927, 5)
sports_facility.geojson 로드 완료. 데이터 크기: (903, 4)
trash_bin_2024.geojson 로드 완료. 데이터 크기: (5380, 4)
파일명: bus_stops.geojson, 열 목록: ['노드 ID', '정류소번호', '정류소명', 'longitude', 'latitude', '정류소 타입', 'geometry']
파일명: legal_boundary.geojson, 열 목록: ['SIG_CD', 'SIG_KOR_NM', 'geometry']
파일명: parking_lot.geojson, 열 목록: ['주소', 'latitude', 'longitude', 'geometry']
파일명: road.geojson, 열 목록: ['BSI_INT', 'RBP_CN', 'RDS_DPN_SE', 'RDS_MAN_NO', 'REP_CN', 'RN', 'RN_CD', 'ROAD_BT', 'ROAD_LT', 'ROA_CLS_SE', 'WDR_RD_CD', 'geometry']
파일명: school.geojson, 열 목록: ['학교명', '도로명주소', 'latitude', 'longitude', 'geometry']
파일명: sports_facility.geojson, 열 목록: ['시설주소', 'latitude', 'longitude', 'geometry']
파일명: trash_bin_2024.geojson, 열 목록: ['address', 'latitude', 'longitude', 'geometry']


In [3]:
map_folium = folium.Map(location=[37.5665, 126.9780], tiles="OpenStreetMap", zoom_start=12)

# FeatureGroup 생성
fg = folium.FeatureGroup(name='ALL')
map_folium.add_child(fg)


In [5]:

# 공통 MarkerCluster 옵션 (기본 줌 제한: 15)
default_marker_cluster_options = {
    "zoomToBoundsOnClick": True,      # 클러스터 클릭 시 확대
    "showCoverageOnHover": True,      # 마우스 오버 시 클러스터 영역 표시
    "maxClusterRadius": 200,          # 클러스터링 반경 (픽셀)
    "disableClusteringAtZoom": 15     # 줌 레벨 15 이상에서 클러스터링 비활성화, 14는 범위가 너무 큰듯
}

# 버스 정류장 MarkerCluster 옵션 (줌 제한: 18)
bus_stop_marker_cluster_options = {
    "zoomToBoundsOnClick": True,
    "showCoverageOnHover": True,
    "maxClusterRadius": 200,
    "disableClusteringAtZoom": 18  # 줌 레벨 18 이상에서 클러스터링 비활성화
}

def add_layer(map_object = map_folium, parent_group = fg, layer_name = None, data=None, popup_field=None, color=None, icon=None, cluster_options=default_marker_cluster_options):
    """
    특정 데이터를 Folium 지도에 레이어로 추가하는 함수.

    Parameters:
        map_object (folium.Map): Folium 지도 객체.
        parent_group (folium.FeatureGroup): 상위 FeatureGroup.
        layer_name (str): 레이어 이름.
        data (pd.DataFrame): 추가할 데이터.
        popup_field (str): 팝업에 표시할 데이터 칼럼 이름.
        color (str): 마커 색상.
        icon (str): 마커 아이콘.
        cluster_options (dict): MarkerCluster 옵션.
    """
    # FeatureGroupSubGroup 생성
    group = folium.plugins.FeatureGroupSubGroup(parent_group, name=layer_name)
    map_object.add_child(group)
    
    # MarkerCluster 추가
    marker_cluster = MarkerCluster(options=cluster_options).add_to(group)
    
    # 데이터 순회하며 마커 추가
    for _, row in data.iterrows():
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=folium.Popup(f"{layer_name}: {row[popup_field]}", max_width=300),
            icon=folium.Icon(color=color, icon=icon,
                             prefix='fa'
            )
        ).add_to(marker_cluster)
    return group
# 함수화 후 레이어 추가
g1 = add_layer(layer_name='버스 정류장',
               data = geo_df['bus_stops.geojson'],
               popup_field ='정류소명',
               color = 'purple',
               icon = 'bus',
               cluster_options=bus_stop_marker_cluster_options,
              )

g2 = add_layer(layer_name='주차장',
               data = geo_df['parking_lot.geojson'],
               popup_field='주소',
               color='black',
               icon = 'car',
               cluster_options = bus_stop_marker_cluster_options,
               )

g3 = add_layer(layer_name = '학교',
               data = geo_df['school.geojson'],
               popup_field = '학교명',
               color = 'green',
               icon = 'book',
               )

g4 = add_layer(layer_name='체육 시설',
               data = geo_df['sports_facility.geojson'],
               popup_field='시설주소',
               color = 'orange',
               icon = 'bicycle',
                )

g4 = add_layer(layer_name ='기존 쓰레기통',
               data = geo_df['trash_bin_2024.geojson'],
               popup_field ='address',
               color = 'red',
               icon = 'trash',
               )

# legal_boundary 데이터 (구 경계 데이터) 추가
folium.GeoJson(
    geo_df['legal_boundary.geojson'],
    name="구",  # 레이어 이름
    style_function=lambda feature: {
        'fillColor': '#00b493',        # 영역 채우기 색상 (은색)
        'color': '#007c65',             # 경계선 색상
        'weight': 2,                  # 경계선 두께
        'fillOpacity': 0.2            # 영역 투명도
    },
    tooltip=folium.GeoJsonTooltip(fields=['SIG_KOR_NM'], aliases=['구:'])  # 팁으로 구 이름 표시
).add_to(map_folium)

# 레이어 컨트롤 추가
folium.LayerControl(collapsed=False).add_to(map_folium)

# 결과 저장
map_folium.save("./result/test6.html")
print("지도 저장 완료: test6.html")

지도 저장 완료: test6.html
